# Spec2Vec API – Quickstart

*In order to use the client, you need an access token provided by DataRevenue.*
## Introduction

This is a short guide on you can add GNPS library matches to your mass spectra - using Omigami's Spec2Vec API.

The API uses a Spec2Vec model that was trained on the entire GNPS spectral library. It embeds each of your spectra into a vector space and calculates the cosine similarity to all GNPS library spectra in this vector space. It then returns you the top library matches for each of your spectra. To learn more about Spec2Vec read our gentle introduction: [Spec2Vec: The Next Step in Mass Spectral Similarity Metrics](https://www.datarevenue.com/en-blog/spec2vec-mass-spectral-similarity-metric)

This notebook shows you how 
1. Specify any MGF file
2. Run a library search through the Spec2Vec API
3. Save the results as CSV.

# 1. (Down)load a MS/MS dataset (MGF format)

We'll use a small MS/MS dataset in the MGF format from [here](https://gnps-external.ucsd.edu/gnpslibrary/GNPS-COLLECTIONS-MISC.mgf).
You can also select any other from the [GNPS spectral library](https://gnps-external.ucsd.edu/gnpslibrary), or - of course - use your own.

*Note that you will need the Precursor_MZ field `PEPMASS` and the abundance pairs in your MGF file.*

In [ ]:
# Load your own MS/MS dataset (and skip the next cell)
path_to_mgf <- '/path/to/local_dataset.mgf'

In [6]:
# OR download a small MS/MS dataset from GNPS, in the same directory as this notebook
url = 'https://gnps-external.ucsd.edu/gnpslibrary/GNPS-COLLECTIONS-MISC.mgf'

path_to_mgf = 'GNPS-COLLECTIONS-MISC.mgf' # use your prefered saving path here

download.file(url, path_to_mgf, method = "curl")

# 2. Query for the best matches with Spec2Vec

`Romigami` is a R wrapper which instantiates a virtual environment on which to install and call the Python `Omigami` package

`Omigami` is a Python wrapper which :
- Builds a json payload from the MGF file
- Calls the Spec2Vec API
- Formats the prediction results into readable dataframes

____
- `n_best` sets the number of matches you'd like per spectrum (it is set to 10 by default).

In the results dataframes, the input spectra can be identified by their number in the dataframes index, which refers to their order in the MGF file.  
*i.e.* `matches of spectrum 1` gives the spectrum_id and Spec2Vec scores of the library spectra matches, for the first spectrum in the MGF file.

For each spectrum in the MGF file, the library spectra matches are sorted according to their Spec2Vec similarity score (best is first).   
The following information about the predicted spectra are returned :
- `score`, the Spec2Vec similarity score between the input spectrum and the library spectrum
- `matches of spectrum #`, the spectrum_ID of the matched library spectra for the spectrum number # in the MGF file

In [20]:
# import the package
library("romigami")


Attachement du package : ‘romigami’


Les objets suivants sont masqués _par_ ‘.GlobalEnv’:

    initialize_environment, match_spectra_from_path




In [21]:
# environment setup
initialize_environment()

virtualenv: Romigami
Using virtual environment '/Users/pierre/.virtualenvs/Romigami' ...


In [19]:
# Run Spec2Vec library search with your user token
spectra_matches <- match_spectra_from_path(token = "8DmWwNrFB8oSQcjHsEabHoNcUy30MZLY",
                        mgf_path = path_to_mgf,
                        n_best = 10)

ERROR: Error in py_call_impl(callable, dots$args, dots$keywords): InvalidCredentials: Your credentials are invalid, please revise your API token.

Detailed traceback:
  File "/Users/pierre/.virtualenvs/Romigami/lib/python3.9/site-packages/omigami/spec2vec.py", line 60, in match_spectra_from_path
    requests.append(self._send_request(payload))
  File "/Users/pierre/.virtualenvs/Romigami/lib/python3.9/site-packages/omigami/spec2vec.py", line 158, in _send_request
    raise InvalidCredentials(



## 2.1 View results
A list of dataframes is returned. To look at a specific dataframe you can call :
```python
spectra_matches[i]  # 'i' refers to the ith spectrum in the MGF file input
```

In [11]:
spectra_matches[1]

,score
,<dbl>
CCMSLIB00006015264,0.2317165
CCMSLIB00005914380,0.2210768
CCMSLIB00006056045,0.1919893
CCMSLIB00005914391,0.1896599
CCMSLIB00005914553,0.1896599
CCMSLIB00006015384,0.1830748
CCMSLIB00005987253,0.1587133
CCMSLIB00006056696,0.1577501
CCMSLIB00006015008,0.1552923


# 3. Save results

Execute the following cell to save the results in a CSV file. For readability, each dataframe is saved in its own CSV file under the `matches` directory.

In [18]:
if (!dir.exists("matches")){
    dir.create("matches")
}
for (i in seq_along(spectra_matches)){
    write.csv(spectra_matches[[i]],sprintf("matches/spectrum_%s.csv",i))
}


___